# Monday.com Timesheet Aggregated Version
-------

# Import Libraries

In [ ]:
import requests
import json
import csv
from monday import MondayClient
import openpyxl
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
import datetime
from datetime import timedelta
import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config['DEFAULT']['API-Key']
timesheet_date = config['DEFAULT']['Timesheet-Date']
board_ids = config['DEFAULT']['Board-ID-List']
date = config['DEFAULT']['Current-Date']
time_difference = config['DEFAULT']['Time-Difference']
change_status = config['DEFAULT']['Change-Status']

# Supporting Functions

In [ ]:
def digit_or_dot(s):
    return all(c.isdigit() or c == '.' for c in s)

In [ ]:
def get_congif_info(person):
    surname = config[person]['Surname']
    given_names = config[person]['Given-Names']
    employee_number = config[person]['Employee-Number']
    dob = config[person]['Date-Of-Birth']
    phone_number = config[person]['Phone-Number']
    rate = config[person]['Rate']
    pay_code = config[person]['Pay-Code']
    
    return surname,given_names,employee_number,dob,phone_number,rate,pay_code

In [ ]:
def create_cell(column, text, align):
    sheet["{}{}".format(column,row_num)] = text
    sheet["{}{}".format(column,row_num)].font = table_font
    sheet["{}{}".format(column,row_num)].border = thin_full_border
    if align == 1:
        sheet["{}{}".format(column,row_num)].alignment = Alignment(wrap_text = True, horizontal='center', vertical='center')
    else:
        sheet["{}{}".format(column,row_num)].alignment = Alignment(wrap_text = True, vertical='center')

In [ ]:
def insert_new_row(row_num):
    sheet.insert_rows(row_num)
    for col in range(1,12):
        sheet.row_dimensions[row_num].height = 25
        sheet.cell(row=row_num, column=col).border = thin_full_border
        sheet.cell(row=row_num, column=col).fill = grey_fill

# Get Person List & Project Code List

In [ ]:
person_list = []
project_code_list = {}
for s in config.sections():
    if digit_or_dot(s):
        project_code_list[s] = {}
    else:
        person_list.append(s)

In [ ]:
for pc in project_code_list:
    project_code_list[pc]["name"] = config[pc]['Name']
    project_code_list[pc]["department_name"] = config[pc]['Department-Name']
    project_code_list[pc]["phone"] = config[pc]['Phone']
    project_code_list[pc]["date"] = config[pc]['Date']
    project_code_list[pc]["subject_code"] = config[pc]['Subject-Code']
    project_code_list[pc]["board_ids"] = []

# Connect To Monday.com

In [ ]:
apiUrl = "https://api.monday.com/v2"
headers = {"Authorization" : api_key}
monday = MondayClient(api_key)

# Get Board ID List

In [ ]:
board_id_list = []
if board_ids:
    for b in board_ids.split(","):
        board_id_list.append(b)
else:
    query = '{ boards {id}}'
    data = {'query' : query}
    r = requests.post(url=apiUrl, json=data, headers=headers)
    for bd in r.json()["data"]["boards"]:
        board_id_list.append(bd["id"])

# Get Board.Group As Prefix For Job ID

In [ ]:
group_item_dict = {}

for board_id in board_id_list:
    board_query = '{{boards (ids: {}){{description name groups{{id}}}}}}'.format(board_id)
    board_data = {'query' : board_query}
    board_result = requests.post(url=apiUrl, json=board_data, headers=headers)
    
    board_name = board_result.json()["data"]["boards"][0]["name"]
    board_project_code = board_result.json()["data"]["boards"][0]["description"].split("\n")[0].split(": ")[1]
        
    project_code_list[board_project_code]["board_ids"].append(board_id)
    
    for g in board_result.json()["data"]["boards"][0]["groups"]:
        group_query = '{{boards (ids: {}){{groups(ids: {}) {{title items {{id}}}}}}}}'.format(board_id,g["id"])
        group_data = {'query' : group_query}
        group_result = requests.post(url=apiUrl, json=group_data, headers=headers)
        group_title = group_result.json()["data"]["boards"][0]["groups"][0]["title"]
        for i in group_result.json()["data"]["boards"][0]["groups"][0]["items"]:
            group_item_dict[i["id"]] = [board_name,group_title]

# Setup Border & Font & Color

In [ ]:
thin_full_border = Border(left=Side(style='thin'), 
                     right=Side(style='thin'), 
                     top=Side(style='thin'), 
                     bottom=Side(style='thin'))

thin_top_bot_border = Border(top=Side(style='thin'), 
                     bottom=Side(style='thin'))

grey_fill = PatternFill(start_color='F2F2F2',
                   end_color='F2F2F2',
                   fill_type='solid')

table_font = Font(name='Open Sans', size=11)

table_font_authen = Font(bold=True,name='Open Sans', size=12)

# Create Timesheet

In [ ]:
datetimeFormat = '%Y-%m-%dT%H:%M:%SZ'

for p in person_list:
    for pc in project_code_list:
        workday_time_dict = {}
        if p.replace(" ","_") in config[pc]:
            position_id,primary_position = config[pc][p.replace(" ","_")].split(",")
            surname,given_names,employee_number,dob,phone_number,rate,pay_code = get_congif_info(p)
            
            # read template excel
            workbook = openpyxl.load_workbook('template.xlsx')
            sheet = workbook["Timesheet Loader Template"]
            
            # add tasks from Monday.com
            row_num = 6
            
            for b in project_code_list[pc]["board_ids"]:
                query = '{{items_by_column_values (board_id :{}, column_id: "status", column_value:"{}"){{id name column_values{{text title value}}}}}}'.format(b, "Waiting for timesheet")
                data = {'query' : query}
                
                r = requests.post(url=apiUrl, json=data, headers=headers) # make request
                
                for v in r.json()["data"]["items_by_column_values"]:
                    job_number = v["id"]
                    ignore = 1

                    # check if person in the group
                    for cv in v["column_values"]:
                        if cv["title"] == "Person":
                            # only group "meeting" OR "meetings" allows multiple people
                            if group_item_dict[job_number][1].lower() in {"meeting","meetings"}:
                                if p in cv["text"]:
                                    ignore = 0
                                else:
                                    ignore = 1
                                    continue
                            else:
                                if "," in cv["text"]:
                                    print("----------------------------------")
                                    print("Error Group Name / Job Number: {} / {}".format(group_item_dict[job_number][1],job_number))
                                    print("Assigned to persons: {}".format(cv["text"]))
                                    print("Error : multiple persons are assigned to same task & it is not under meeting(s)")
                                    continue
                                else:
                                    if cv["text"] == p:
                                        ignore = 0
                                    else:
                                        ignore = 1
                                        continue

                    if ignore == 0:
                        for cv in v["column_values"]:
                            if cv["title"] == "Time Tracking":
                                time_tracking = cv["text"]
                                if time_tracking == "":
                                    time_used = 0
                                else:
                                    for av in json.loads(cv["value"])["additional_value"]:
                                        # convert date string to datetime
                                        start_at = datetime.datetime.strptime(av["started_at"], datetimeFormat)
                                        end_at = datetime.datetime.strptime(av["ended_at"], datetimeFormat)

                                        # change to correct time zone
                                        start_at += datetime.timedelta(hours=int(time_difference))
                                        end_at += datetime.timedelta(hours=int(time_difference))

                                        work_day = str(end_at)[:10]

                                        # calculate time range
                                        diff = end_at - start_at
                                        time_used = round(diff.seconds/3600, 2)

                                        # aggregate by work day
                                        if work_day in workday_time_dict:
                                            workday_time_dict[work_day] += time_used
                                        else:
                                            workday_time_dict[work_day] = time_used
                        
                        if check_status:
                            # change status to "Done"
                            monday.items.change_item_value(b, int(v["id"]), "status", {"index": 1})

                        # add timesheet created date
                        monday.items.change_item_value(b, int(v["id"]), "date7", {"date":timesheet_date})
                    else:
                        continue
            
            subject_code = project_code_list[pc]["subject_code"]
            
            if check_status:
                if len(workday_time_dict) > 0:
                    for workday in sorted(workday_time_dict):

                        # insert new rows
                        if row_num > 6:
                            insert_new_row(row_num)

                        row_list = [surname, given_names, employee_number, position_id, primary_position, workday, pay_code, workday_time_dict[workday], rate, pc, subject_code]

                        for col, val in enumerate(row_list, start=1):
                            sheet.cell(row=row_num, column=col).value = val
                            sheet.cell(row=row_num, column=col).font = table_font
                            sheet.cell(row=row_num, column=col).alignment = Alignment(wrap_text = True, vertical='center')

                        row_num += 1

                    # add_personal_info
                    sheet.row_dimensions[row_num].height = 25
                    sheet["A{}".format(row_num)] = "AUTHORISATION:"
                    sheet["A{}".format(row_num)].font = table_font_authen
                    sheet["A{}".format(row_num)].alignment = Alignment(wrap_text = True, vertical='center')
                    sheet["A{}".format(row_num)].border = thin_top_bot_border
                    sheet.merge_cells('A{}:K{}'.format(row_num,row_num))

                    row_num += 1
                    sheet.row_dimensions[row_num].height = 25
                    sheet["A{}".format(row_num)] = "Claimant:"
                    sheet["A{}".format(row_num)].font = table_font_authen
                    sheet["A{}".format(row_num)].border = thin_full_border
                    sheet["A{}".format(row_num)].alignment = Alignment(wrap_text = True, vertical='center')
                    sheet.merge_cells('A{}:K{}'.format(row_num,row_num))

                    row_num += 1
                    sheet.row_dimensions[row_num].height = 25
                    create_cell("A", "Name:", 0)

                    create_cell("B", "{} {}".format(given_names, surname), 1)
                    sheet.merge_cells('B{}:G{}'.format(row_num,row_num))

                    create_cell("H", "Name:", 0)
                    create_cell("I", project_code_list[pc]["name"], 1)
                    create_cell("J", "Department Name:", 0)
                    create_cell("K", project_code_list[pc]["department_name"], 1)

                    row_num += 1
                    sheet.row_dimensions[row_num].height = 25
                    create_cell("A", "Phone:", 0)

                    create_cell("B", phone_number, 1)
                    sheet.merge_cells('B{}:C{}'.format(row_num,row_num))

                    create_cell("D", "Date:", 0)

                    create_cell("E", date, 1)
                    sheet.merge_cells('E{}:G{}'.format(row_num,row_num))

                    create_cell("H", "Phone:", 0)
                    create_cell("I", project_code_list[pc]["phone"], 1)
                    create_cell("J", "Date:", 0)
                    create_cell("K", project_code_list[pc]["date"], 1)

                    #save changes in excel
                    workbook.save(filename="{}_{}_{}({}).xlsx".format(given_names,surname,timesheet_date.replace('-','_'),pc))
                    time.sleep(5)
                else:
                    continue
            else:
                continue